In [6]:
import numpy as np
import time
np.random.seed(42)

In [7]:
def transpose(matrix):
    return [list(row) for row in zip(*matrix)]

def matmul(A, B):
    result = []
    for i in range(len(A)):
        row = []
        for j in range(len(B[0])):
            row.append(sum(A[i][k] * B[k][j] for k in range(len(A[0]))))
        result.append(row)
    return result

def identity_matrix(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]

def diagonal_inv(sigma, tol=1e-10):
    return [[1/sigma[i] if i == j and sigma[i] > tol else 0 for j in range(len(sigma))] for i in range(len(sigma))]

def norm(v):
    return sum(x**2 for x in v) ** 0.5

def dot(u, v):
    return sum(u[i]*v[i] for i in range(len(u)))

def subtract(u, v):
    return [u[i] - v[i] for i in range(len(u))]

def scalar_multiply(s, v):
    return [s * x for x in v]

def gram_schmidt(A):
    Q = []
    for a in A:
        u = a[:]
        for q in Q:
            proj = scalar_multiply(dot(u, q), q)
            u = subtract(u, proj)
        u_norm = norm(u)
        if u_norm > 1e-10:
            Q.append(scalar_multiply(1/u_norm, u))
    return Q

def eigen_decomposition_sym(A, num_iter=100):
    n = len(A)
    V = identity_matrix(n)
    for _ in range(num_iter):
        for p in range(n):
            for q in range(p+1, n):
                if abs(A[p][q]) < 1e-10:
                    continue
                theta = 0.5 * np.arctan2(2*A[p][q], A[q][q] - A[p][p])
                cos = np.cos(theta)
                sin = np.sin(theta)

                for i in range(n):
                    Api = A[i][p]
                    Aiq = A[i][q]
                    A[i][p] = cos * Api - sin * Aiq
                    A[i][q] = sin * Api + cos * Aiq

                for j in range(n):
                    Apj = A[p][j]
                    Aqj = A[q][j]
                    A[p][j] = cos * Apj - sin * Aqj
                    A[q][j] = sin * Apj + cos * Aqj

                for i in range(n):
                    Vip = V[i][p]
                    Viq = V[i][q]
                    V[i][p] = cos * Vip - sin * Viq
                    V[i][q] = sin * Vip + cos * Viq
    eigenvalues = [A[i][i] for i in range(n)]
    return eigenvalues, V

def svd_inverse(A):
    A_T = transpose(A)
    ATA = matmul(A_T, A)
    eigvals, V = eigen_decomposition_sym([row[:] for row in ATA])
    
    sigma = [eigval**0.5 if eigval > 0 else 0 for eigval in eigvals]
    sigma_inv = diagonal_inv(sigma)

    V_T = transpose(V)

    AV = matmul(A, V)
    U = []
    for i in range(len(AV[0])):
        col = [AV[j][i] for j in range(len(AV))]
        s = sigma[i]
        if s > 1e-10:
            U.append([x / s for x in col])
        else:
            U.append([0.0] * len(AV))
    U_T = transpose(U)

    temp = matmul(V, sigma_inv)
    A_inv = matmul(temp, U_T)
    return A_inv

def generate_random_matrix(n):
    return np.random.randint(1, 10, (n, n)).tolist()


matrix_sizes = [10, 100, 1000]

for size in matrix_sizes:
    A = generate_random_matrix(size)
    start_time = time.time()
    A_inv = svd_inverse(A)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"\nSize {size}x{size}: Execution time = {execution_time:.6f} seconds")
    print(f"First 3 rows of inverse matrix {size}x{size}:")
    for row in A_inv[:3]:
        formatted_row = [round(float(val), 4) for val in row]
        print(formatted_row)



Size 10x10: Execution time = 0.004368 seconds
First 3 rows of inverse matrix 10x10:
[-0.077, 0.1173, 0.1318, -0.036, -0.1306, 0.0203, 0.0626, 0.0094, -0.1005, -0.0614]
[0.0718, 0.2307, -0.0934, -0.2316, -0.0017, 0.0461, -0.1352, 0.0845, 0.0117, 0.0838]
[-0.0637, -0.0583, 0.0613, -0.0309, -0.0225, -0.0181, -0.0672, -0.007, -0.0304, 0.0329]

Size 100x100: Execution time = 2.980789 seconds
First 3 rows of inverse matrix 100x100:
[-0.0087, -0.0237, 0.0293, 0.0116, -0.0302, 0.0317, 0.0273, -0.0084, 0.006, 0.05, 0.0038, -0.0009, -0.0029, 0.0091, -0.0158, 0.015, -0.0061, 0.0437, -0.0136, 0.037, 0.0111, 0.0135, -0.0088, -0.0322, 0.0089, 0.0015, -0.0403, 0.0263, -0.0295, -0.014, 0.0219, 0.0376, -0.0005, 0.042, -0.0307, -0.0577, -0.032, -0.0228, -0.0187, 0.0216, -0.0167, 0.0177, -0.0167, 0.0127, 0.0279, -0.0051, -0.021, -0.05, 0.02, -0.0166, -0.0214, -0.0693, 0.0235, 0.0096, 0.0187, 0.0306, -0.0514, 0.0124, 0.0332, -0.0168, 0.0191, 0.0263, -0.0292, -0.0453, 0.0023, 0.0319, -0.024, -0.0281, -0.0

KeyboardInterrupt: 